In [14]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

In [15]:
articles = pd.read_csv('Gaza_textcontain_after_new.csv')
authors = pd.read_csv('Gaza_textcontain_after_new_authors.csv')

In [16]:
articles

,article_title,author,published_time,article_text,article_category_one,article_category_two,picture_description
0,Palestinian death toll in Gaza from Israeli at...,By Xinhua,"Published: Mar 07, 2024 10:34 AM",A young man walks past destroyed buildings in ...,PHOTO,WORLD,A young man walks past destroyed buildings in ...
1,Daily life at Al-Maghazi refugee camp in Gaza ...,By Xinhua,"Published: Mar 07, 2024 11:44 AM",A vendor makes desserts at Al-Maghazi refugee ...,PHOTO,WORLD,A vendor makes desserts at Al-Maghazi refugee ...
2,Two-State solution only way to break vicious c...,By Global Times,"Published: Mar 07, 2024 12:50 PM",People walk past destroyed buildings in the so...,CHINA,POLITICS,People walk past destroyed buildings in the so...
3,Biden to deliver State of the Union address,By GT staff reporters,"Published: Mar 07, 2024 10:28 PM",US President Joe Biden speaks during a meeting...,CHINA,DIPLOMACY,US President Joe Biden speaks during a meeting...
4,Houthi attacks in Red Sea reports first fatali...,By Fan Anqi,"Published: Mar 07, 2024 10:58 PM",A container ship sails in the Red Sea in the S...,CHINA,DIPLOMACY,A container ship sails in the Red Sea in the S...
...,...,...,...,...,...,...,...
485,Political efforts must be intensified to solve...,By Xie Wenting and Bai Yunyi,"Published: Oct 22, 2023 06:32 PM",Teams conduct a search and rescue operation un...,IN-DEPTH,DIPLOMATIC CHANNEL,Teams conduct a search and rescue operation un...
486,US biased attitude in Israel-Palestine conflic...,By Ma Xiaolin,"Published: Oct 22, 2023 08:21 PM",Illustration: Liu Rui/ GTAny act that leads to...,OPINION,VIEWPOINT,Illustration: Liu Rui/ GT
487,US inability to reconcile Israel-Palestine con...,By Global Times,"Published: Oct 22, 2023 08:32 PM","NUSEIRAT, GAZA - OCTOBER 22: A view of destruc...",OPINION,OBSERVER,"NUSEIRAT, GAZA - OCTOBER 22: A view of destruc..."
488,Israel’s deepening attacks in Gaza likely to e...,Zhao Yusha,"Published: Oct 22, 2023 09:31 PM Updated: Oct ...",Leaders and top officials from more than a doz...,CHINA,DIPLOMACY,Leaders and top officials from more than a doz...


In [17]:
# date published
#remoe where published time = published time not found

articles = articles.drop(articles[articles['published_time'] == 'Published time not found'].index)

articles['published_time'] = articles['published_time'].str[11:]
articles['published_time'] = articles['published_time'].str[:12]
#to date mmm dd, yyyy
articles['published_time'] = pd.to_datetime(articles['published_time'], format='%b %d, %Y')
articles

,article_title,author,published_time,article_text,article_category_one,article_category_two,picture_description
0,Palestinian death toll in Gaza from Israeli at...,By Xinhua,2024-03-07,A young man walks past destroyed buildings in ...,PHOTO,WORLD,A young man walks past destroyed buildings in ...
1,Daily life at Al-Maghazi refugee camp in Gaza ...,By Xinhua,2024-03-07,A vendor makes desserts at Al-Maghazi refugee ...,PHOTO,WORLD,A vendor makes desserts at Al-Maghazi refugee ...
2,Two-State solution only way to break vicious c...,By Global Times,2024-03-07,People walk past destroyed buildings in the so...,CHINA,POLITICS,People walk past destroyed buildings in the so...
3,Biden to deliver State of the Union address,By GT staff reporters,2024-03-07,US President Joe Biden speaks during a meeting...,CHINA,DIPLOMACY,US President Joe Biden speaks during a meeting...
4,Houthi attacks in Red Sea reports first fatali...,By Fan Anqi,2024-03-07,A container ship sails in the Red Sea in the S...,CHINA,DIPLOMACY,A container ship sails in the Red Sea in the S...
...,...,...,...,...,...,...,...
485,Political efforts must be intensified to solve...,By Xie Wenting and Bai Yunyi,2023-10-22,Teams conduct a search and rescue operation un...,IN-DEPTH,DIPLOMATIC CHANNEL,Teams conduct a search and rescue operation un...
486,US biased attitude in Israel-Palestine conflic...,By Ma Xiaolin,2023-10-22,Illustration: Liu Rui/ GTAny act that leads to...,OPINION,VIEWPOINT,Illustration: Liu Rui/ GT
487,US inability to reconcile Israel-Palestine con...,By Global Times,2023-10-22,"NUSEIRAT, GAZA - OCTOBER 22: A view of destruc...",OPINION,OBSERVER,"NUSEIRAT, GAZA - OCTOBER 22: A view of destruc..."
488,Israel’s deepening attacks in Gaza likely to e...,Zhao Yusha,2023-10-22,Leaders and top officials from more than a doz...,CHINA,DIPLOMACY,Leaders and top officials from more than a doz...


In [18]:
#stats published time
articles['author'].unique()

array(['By Xinhua', 'By Global Times', 'By GT staff reporters',
       'By Fan Anqi', 'By Mark Blacklock', 'By VCG', 'By Xin Ping',
       'By Zhang Han', 'Shan Jie', 'By Brian Berletic', 'Yang Sheng',
       'By S.L. Kanthan', 'By Ma Yuhan', 'By Elias Cepeda', 'Zhao Yusha',
       'By Chen Hong', 'By IC', 'Bradley Blankenship', 'Wang Qi',
       'By Zhao Yongsheng', 'By Wang Shushen', 'By Timofei Bordachev',
       'By Bai Yunyi', 'Chen Qingqing', 'Ding Gang', 'By Zhang Wenzong',
       'By James Smith', 'By Xu Yelu', 'By Xia Wenxin', 'Andrey Kortunov',
       'By Wang Jin', 'By Xie Wenting and Bai Yunyi', 'Wang Cong',
       'Deng Xiaoci', 'By Joyce Chimbi', 'By Andrey Gubin',
       'By Victor Gao', 'By AFP', 'By Wang Yan and Zhang Yuanshi',
       'By Qian Jiayin', 'Hu Xijin', 'By Jerry Grey', 'By Jin Cuodao',
       'By Yin Yeping', 'By Jan Oberg', 'Vitaly Podvitski', 'Liu Xin',
       'Liu Caiyu', 'By Ma Ruiqian', 'By Ebrahim Hashem', 'By Guo Yichu',
       'Martin Jacques', 'By 

In [19]:
# celan authors
articles['author'] = articles['author'].apply(lambda x: x[3:] if x.startswith("By ") else x)
articles

,article_title,author,published_time,article_text,article_category_one,article_category_two,picture_description
0,Palestinian death toll in Gaza from Israeli at...,Xinhua,2024-03-07,A young man walks past destroyed buildings in ...,PHOTO,WORLD,A young man walks past destroyed buildings in ...
1,Daily life at Al-Maghazi refugee camp in Gaza ...,Xinhua,2024-03-07,A vendor makes desserts at Al-Maghazi refugee ...,PHOTO,WORLD,A vendor makes desserts at Al-Maghazi refugee ...
2,Two-State solution only way to break vicious c...,Global Times,2024-03-07,People walk past destroyed buildings in the so...,CHINA,POLITICS,People walk past destroyed buildings in the so...
3,Biden to deliver State of the Union address,GT staff reporters,2024-03-07,US President Joe Biden speaks during a meeting...,CHINA,DIPLOMACY,US President Joe Biden speaks during a meeting...
4,Houthi attacks in Red Sea reports first fatali...,Fan Anqi,2024-03-07,A container ship sails in the Red Sea in the S...,CHINA,DIPLOMACY,A container ship sails in the Red Sea in the S...
...,...,...,...,...,...,...,...
485,Political efforts must be intensified to solve...,Xie Wenting and Bai Yunyi,2023-10-22,Teams conduct a search and rescue operation un...,IN-DEPTH,DIPLOMATIC CHANNEL,Teams conduct a search and rescue operation un...
486,US biased attitude in Israel-Palestine conflic...,Ma Xiaolin,2023-10-22,Illustration: Liu Rui/ GTAny act that leads to...,OPINION,VIEWPOINT,Illustration: Liu Rui/ GT
487,US inability to reconcile Israel-Palestine con...,Global Times,2023-10-22,"NUSEIRAT, GAZA - OCTOBER 22: A view of destruc...",OPINION,OBSERVER,"NUSEIRAT, GAZA - OCTOBER 22: A view of destruc..."
488,Israel’s deepening attacks in Gaza likely to e...,Zhao Yusha,2023-10-22,Leaders and top officials from more than a doz...,CHINA,DIPLOMACY,Leaders and top officials from more than a doz...


In [20]:
articles['article_text'] = articles.apply(lambda x: x['article_text'].replace(x['picture_description'], ''), axis=1)
articles

,article_title,author,published_time,article_text,article_category_one,article_category_two,picture_description
0,Palestinian death toll in Gaza from Israeli at...,Xinhua,2024-03-07,\n\nPeople walk past destroyed buildings in t...,PHOTO,WORLD,A young man walks past destroyed buildings in ...
1,Daily life at Al-Maghazi refugee camp in Gaza ...,Xinhua,2024-03-07,\n\nChildren are pictured at Al-Maghazi refug...,PHOTO,WORLD,A vendor makes desserts at Al-Maghazi refugee ...
2,Two-State solution only way to break vicious c...,Global Times,2024-03-07,There is no distinction between noble and humb...,CHINA,POLITICS,People walk past destroyed buildings in the so...
3,Biden to deliver State of the Union address,GT staff reporters,2024-03-07,US President Joe Biden is expected to deliver ...,CHINA,DIPLOMACY,US President Joe Biden speaks during a meeting...
4,Houthi attacks in Red Sea reports first fatali...,Fan Anqi,2024-03-07,A Houthi missile attack on a merchant ship in ...,CHINA,DIPLOMACY,A container ship sails in the Red Sea in the S...
...,...,...,...,...,...,...,...
485,Political efforts must be intensified to solve...,Xie Wenting and Bai Yunyi,2023-10-22,Editor's Note:The death toll among Palestinian...,IN-DEPTH,DIPLOMATIC CHANNEL,Teams conduct a search and rescue operation un...
486,US biased attitude in Israel-Palestine conflic...,Ma Xiaolin,2023-10-22,Any act that leads to civilian deaths and othe...,OPINION,VIEWPOINT,Illustration: Liu Rui/ GT
487,US inability to reconcile Israel-Palestine con...,Global Times,2023-10-22,US media is currently attempting to redirect t...,OPINION,OBSERVER,"NUSEIRAT, GAZA - OCTOBER 22: A view of destruc..."
488,Israel’s deepening attacks in Gaza likely to e...,Zhao Yusha,2023-10-22,Israel's deepening attacks in Gaza for what it...,CHINA,DIPLOMACY,Leaders and top officials from more than a doz...


In [21]:
df = pd.merge(articles, authors, left_on='author', right_on='author_name', how='left')
df = df.drop(columns=['author_name'])
df.replace(pd.NA, 'Author detiles not found', inplace=True)
df.to_csv('gaza_textcontain_after_new_preprocessed.csv')
df

,article_title,author,published_time,article_text,article_category_one,article_category_two,picture_description,author_title,author_description
0,Palestinian death toll in Gaza from Israeli at...,Xinhua,2024-03-07,\n\nPeople walk past destroyed buildings in t...,PHOTO,WORLD,A young man walks past destroyed buildings in ...,Author detiles not found,Author detiles not found
1,Daily life at Al-Maghazi refugee camp in Gaza ...,Xinhua,2024-03-07,\n\nChildren are pictured at Al-Maghazi refug...,PHOTO,WORLD,A vendor makes desserts at Al-Maghazi refugee ...,Author detiles not found,Author detiles not found
2,Two-State solution only way to break vicious c...,Global Times,2024-03-07,There is no distinction between noble and humb...,CHINA,POLITICS,People walk past destroyed buildings in the so...,Author detiles not found,Author detiles not found
3,Biden to deliver State of the Union address,GT staff reporters,2024-03-07,US President Joe Biden is expected to deliver ...,CHINA,DIPLOMACY,US President Joe Biden speaks during a meeting...,Author detiles not found,Author detiles not found
4,Houthi attacks in Red Sea reports first fatali...,Fan Anqi,2024-03-07,A Houthi missile attack on a merchant ship in ...,CHINA,DIPLOMACY,A container ship sails in the Red Sea in the S...,Author detiles not found,Author detiles not found
...,...,...,...,...,...,...,...,...,...
1203,"GT Exclusive: Mainland tax, natural resource a...",Liu Xin,2023-10-22,Chinese mainland tax authorities have conducte...,CHINA,POLITICS,Foxconn Photo: VCG,"Reporter, Beijing",Liu Xin co-leads the Global Times China desk. ...
1204,"GT Exclusive: Mainland tax, natural resource a...",Liu Xin,2023-10-22,Chinese mainland tax authorities have conducte...,CHINA,POLITICS,Foxconn Photo: VCG,"Reporter, Beijing",Liu Xin co-leads the Global Times China desk. ...
1205,"GT Exclusive: Mainland tax, natural resource a...",Liu Xin,2023-10-22,Chinese mainland tax authorities have conducte...,CHINA,POLITICS,Foxconn Photo: VCG,"Reporter, Beijing",Liu Xin co-leads the Global Times China desk. ...
1206,"GT Exclusive: Mainland tax, natural resource a...",Liu Xin,2023-10-22,Chinese mainland tax authorities have conducte...,CHINA,POLITICS,Foxconn Photo: VCG,"Reporter, Beijing",Liu Xin co-leads the Global Times China desk. ...


In [22]:
df['article_category_one'].describe()

count      1208
unique        9
top       CHINA
freq        772
Name: article_category_one, dtype: object

In [10]:
#senance count
df['sentence_count'] = df['article_text'].apply(lambda x: len(x.split('.')) - 1)

In [11]:
#word count using tokenization
#remove stopwords
stop_words = set(stopwords.words('english'))

In [12]:
#word count using tokenization
df['word_count'] = df['article_text'].apply(lambda x: len([word for word in word_tokenize(x) if word.isalpha()]))

In [13]:
# part of speech count using tokenization wihout stopwords
nltk.download('averaged_perceptron_tagger')
df['pos_count'] = df['article_text'].apply(lambda x: FreqDist([tag for word, tag in nltk.pos_tag(word_tokenize(x)) if word.isalpha() and word not in stop_words]))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


KeyboardInterrupt: 

In [ ]:
df['pos_count'].tolist()

In [ ]:
#pos count to data frame
df_pos = pd.DataFrame(df['pos_count'].tolist())
df_pos.fillna(0, inplace=True)
df_pos['sentence_count'] = df['sentence_count']
df_pos['word_count'] = df['word_count']
df_pos

## Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.graph_objects as go


In [ ]:
#word count; cut at 1000
plt.figure(figsize=(12, 10))
sns.histplot(df['word_count'], bins=20, kde=True)
plt.xlim(0, 2000)
#mean line
plt.axvline(df['word_count'].mean(), color='red', linestyle='dashed', linewidth=2, label='Mean')
plt.legend()
plt.title('Word Count Distribution')
plt.show()

In [ ]:
#sentence count; cut at 100
plt.figure(figsize=(12, 10))
sns.histplot(df['sentence_count'], bins=20, kde=True)
plt.xlim(0, 100)
#mean line
plt.axvline(df['sentence_count'].mean(), color='red', linestyle='dashed', linewidth=2, label='Mean')
plt.legend()
plt.title('Sentence Count Distribution')
plt.show()


In [ ]:
#wordcloud form freqdist
fdist = FreqDist([word for word in word_tokenize(' '.join(df['article_text'])) if word.isalpha() and word not in stop_words])
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(fdist.most_common(100)))
plt.figure(figsize=(12, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words')
plt.show()

In [ ]:
#top 10 most common part of speech
plt.figure(figsize=(12, 10))
df_pos.drop(columns=['sentence_count', 'word_count']).sum().sort_values(ascending=False).head(10).plot(kind='bar')
plt.title('Top 10 Most Common Part of Speech')
#turn x labels
plt.xticks(rotation=45)
plt.show()

### Total POS

In [ ]:
NN = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text']))) if tag =="NN" and word.isalpha() and word not in stop_words])
NNP = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text']))) if tag =="NNP" and word.isalpha() and word not in stop_words])
JJ = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text']))) if tag =="JJ" and word.isalpha() and word not in stop_words])
NNS = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text']))) if tag =="NNS" and word.isalpha() and word not in stop_words])
VB = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text']))) if tag =="VB" and word.isalpha() and word not in stop_words])
RB = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text']))) if tag =="RB" and word.isalpha() and word not in stop_words])

In [ ]:
plt.figure(figsize=(12, 18))
plt.subplot(3, 2, 1)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NN.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NN')


plt.subplot(3, 2, 2)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NNP.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NNP')


plt.subplot(3, 2, 3)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(JJ.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of JJ')


plt.subplot(3, 2, 4)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NNS.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NNS')


plt.subplot(3, 2, 5)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(RB.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of VB')

plt.subplot(3, 2, 6)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NN.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of RB')


plt.show()


In [ ]:
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(fdist.most_common(100)))
plt.figure(figsize=(12, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words')
plt.show()

In [ ]:
#sns countplot for article categories
plt.figure(figsize=(12, 10))
sns.countplot(y=df['article_category_one'], order=df['article_category_one'].value_counts().index, color='blue')
plt.title('Article Categories')


In [ ]:
#plot for category one = opinion
#3 plots in a row
plt.figure(figsize=(15, 10))

plt.subplot(1, 3, 1)
sns.countplot(x=df[df['article_category_one'] == 'OPINION']['article_category_two'],  color='blue', order=df[df['article_category_one'] == 'OPINION']['article_category_two'].value_counts().index )
plt.xticks(rotation=45)
plt.ylim(0, 600)
plt.title('OPINION Subcategories')

plt.subplot(1, 3, 2)
sns.countplot(x=df[df['article_category_one'] == 'WORLD']['article_category_two'],  color='blue', order=df[df['article_category_one'] == 'WORLD']['article_category_two'].value_counts().index)
plt.title('WORLD Subcategories')
plt.xticks(rotation=45)
plt.ylim(0, 600)

plt.subplot(1, 3, 3)
sns.countplot(x=df[df['article_category_one'] == 'CHINA']['article_category_two'],  color='blue', order=df[df['article_category_one'] == 'CHINA']['article_category_two'].value_counts().index)
plt.title('CHINA Subcategories')
plt.xticks(rotation=45)
plt.ylim(0, 600)
plt.show()


### POS: diplomacy vs columnists

In [ ]:
NN = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'DIPLOMACY']))) if tag =="NN" and word.isalpha() and word not in stop_words])
NNP = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'DIPLOMACY']))) if tag =="NNP" and word.isalpha() and word not in stop_words])
JJ = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'DIPLOMACY']))) if tag =="JJ" and word.isalpha() and word not in stop_words])
NNS = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'DIPLOMACY']))) if tag =="NNS" and word.isalpha() and word not in stop_words])
VB = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'DIPLOMACY']))) if tag =="VB" and word.isalpha() and word not in stop_words])
RB = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'DIPLOMACY']))) if tag =="RB" and word.isalpha() and word not in stop_words])


In [ ]:
plt.figure(figsize=(6, 36))
plt.subplot(6, 1, 1)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NN.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NN')


plt.subplot(6, 1, 2)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NNP.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NNP')


plt.subplot(6, 1, 3)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(JJ.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of JJ')


plt.subplot(6, 1, 4)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NNS.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NNS')


plt.subplot(6, 1, 5)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(RB.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of VB')

plt.subplot(6, 1, 6)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NN.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of RB')


plt.show()


In [ ]:
NN = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'COLUMNISTS']))) if tag =="NN" and word.isalpha() and word not in stop_words])
NNP = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'COLUMNISTS']))) if tag =="NNP" and word.isalpha() and word not in stop_words])
JJ = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'COLUMNISTS']))) if tag =="JJ" and word.isalpha() and word not in stop_words])
NNS = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'COLUMNISTS']))) if tag =="NNS" and word.isalpha() and word not in stop_words])
VB = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'COLUMNISTS']))) if tag =="VB" and word.isalpha() and word not in stop_words])
RB = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'COLUMNISTS']))) if tag =="RB" and word.isalpha() and word not in stop_words])


In [ ]:
plt.figure(figsize=(6, 36))
plt.subplot(6, 1, 1)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NN.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NN')


plt.subplot(6, 1, 2)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NNP.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NNP')


plt.subplot(6, 1, 3)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(JJ.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of JJ')


plt.subplot(6, 1, 4)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NNS.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NNS')


plt.subplot(6, 1, 5)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(RB.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of VB')

plt.subplot(6, 1, 6)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NN.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of RB')


plt.show()
